# Connexion à l'API France Travail

URL : 

https://francetravail.io/data/api/offres-emploi

https://francetravail.io/data/api/offres-emploi/documentation#/

## Imports

In [134]:
import http.client
import requests
import json
import pandas as pd

In [135]:
# ---------------------------
# CONFIGURATION
# ---------------------------
# France Travail (ex-Pôle emploi)
# FT_CLIENT_ID = Defined in .env
# FT_CLIENT_SECRET = Defined in .env
# FT_SCOPE = Defined in .env

# Paramètres de recherche
JOB_QUERY = "analyst"
LOCATION = "75"

MAX_PAGES = 3   # Limiter le nombre de pages récupérées

# ---------------------------
# AUTH FRANCE TRAVAIL
# ---------------------------
def get_ft_token():
    url = "https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=/partenaire"
    data = {
        "grant_type": "client_credentials",
        "client_id": FT_CLIENT_ID,
        "client_secret": FT_CLIENT_SECRET,
        "scope": FT_SCOPE,
    }
    r = requests.post(url, data=data)
    r.raise_for_status()
    return r.json()["access_token"]

# ---------------------------
# API CALL FRANCE TRAVAIL
# ---------------------------
def fetch_france_travail_jobs(token, max_pages=MAX_PAGES):
    headers = {"Authorization": f"Bearer {token}"}
    all_jobs = []
    for page in range(1, max_pages + 1):
        url = f"https://api.francetravail.io/partenaire/offresdemploi/v2/offres/search"
        params = {
            "motsCles": JOB_QUERY,
            # "lieux": LOCATION,
            "departement": LOCATION,
            "range": f"{(page-1)*50}-{page*50-1}"  # pagination par blocs de 50
        }
        r = requests.get(url, headers=headers, params=params)
        r.raise_for_status()
        data = r.json()
        offres = data.get("resultats", [])
        for o in offres:
            all_jobs.append({
                "source": "France Travail",
                "titre": o.get("intitule"),
                "entreprise": o.get("entreprise", {}).get("nom"),
                "lieu": o.get("lieuTravail", {}).get("libelle"),
                "date": o.get("dateCreation"),
                "url": o.get("origineOffre", {}).get("urlOrigine"),
            })
    return all_jobs

# ---------------------------
# MAIN
# ---------------------------
if __name__ == "__main__":
    print("Authentification France Travail...")
    token = get_ft_token()

    print("Récupération des offres France Travail...")
    ft_jobs = fetch_france_travail_jobs(token)

    df = pd.DataFrame(ft_jobs)
    display(df.head(10))

    # Export vers CSV
    df.to_csv("../data/raw_data/offres_emploi.csv", index=False, encoding="utf-8")
    print(f"{len(ft_jobs)} offres uniques exportées dans offres_emploi.csv ✅")


Authentification France Travail...
Récupération des offres France Travail...


,source,titre,entreprise,lieu,date,url
0,France Travail,Sourcing Analyst -Product development H/F,None,75 - Paris,2025-09-10T15:47:52.147Z,https://candidat.francetravail.fr/offres/reche...
1,France Travail,IT Security Analyst H/F (H/F),ROTHSCHILD & CO MARTIN MAUREL,75 - PARIS 08,2025-09-03T18:04:13.166Z,https://candidat.francetravail.fr/offres/reche...
2,France Travail,AML Analyst - F/H (H/F),LYDIA SOLUTIONS,75 - PARIS 01,2025-09-03T10:08:27.883Z,https://candidat.francetravail.fr/offres/reche...
3,France Travail,Business Analyst F/H (H/F),E-KENT,75 - Paris 19e Arrondissement,2025-08-29T17:18:19.181Z,https://candidat.francetravail.fr/offres/reche...
4,France Travail,Ingénieur(e) Data / Data Analyst (H/F),ODEALIM,75 - PARIS 01,2025-08-29T11:26:55.592Z,https://candidat.francetravail.fr/offres/reche...
5,France Travail,Chargé de Projets Retail & Performance (H/F),None,75 - PARIS 02,2025-09-08T11:22:07.564Z,https://candidat.francetravail.fr/offres/reche...
6,France Travail,Consultant Data Ingénieur (H/F),HEYDATA,75 - PARIS 17,2025-08-19T21:43:37.491Z,https://candidat.francetravail.fr/offres/reche...
7,France Travail,Business Analyst Leasing - Services Financiers...,None,75 - Paris,2025-09-14T10:34:04.000Z,https://candidat.francetravail.fr/offres/reche...
8,France Travail,CLOUD analyst - Azure AWS Finops (H/F),None,75 - Paris,2025-09-14T06:04:09.000Z,https://candidat.francetravail.fr/offres/reche...
9,France Travail,Business analyst (H/F),None,75 - Paris 9e Arrondissement,2025-09-13T17:34:36.000Z,https://candidat.francetravail.fr/offres/reche...


107 offres uniques exportées dans offres_emploi.csv ✅


In [136]:
# # ---------------------------
# # CONFIGURATION
# # ---------------------------
# # France Travail (ex-Pôle emploi)
# # FT_CLIENT_ID = Defined in .env
# # FT_CLIENT_SECRET = Defined in .env
# # FT_SCOPE = Defined in .env

# # Paramètres de recherche
# JOB_QUERY = "data analyst"
# # LOCATION = "75 - Paris"
# LOCATION = {"libelle": "75 - Paris"}

# MAX_PAGES = 3   # Limiter le nombre de pages récupérées

# # ---------------------------
# # AUTH FRANCE TRAVAIL
# # ---------------------------
# def get_ft_token():
#     url = "https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=/partenaire"
#     data = {
#         "grant_type": "client_credentials",
#         "client_id": FT_CLIENT_ID,
#         "client_secret": FT_CLIENT_SECRET,
#         "scope": FT_SCOPE,
#     }
#     r = requests.post(url, data=data)
#     r.raise_for_status()
#     return r.json()["access_token"]

# # ---------------------------
# # API CALL FRANCE TRAVAIL
# # ---------------------------
# def fetch_france_travail_jobs(token, max_pages=MAX_PAGES):
#     headers = {"Authorization": f"Bearer {token}"}
#     all_jobs = []
#     for page in range(1, max_pages + 1):
#         url = f"https://api.francetravail.io/partenaire/offresdemploi/v2/offres/search"
#         params = {
#             "motsCles": JOB_QUERY,
#             # "lieux": LOCATION,
#             "lieuTravail": LOCATION,
#             "range": f"{(page-1)*50}-{page*50-1}"  # pagination par blocs de 50
#         }
#         r = requests.get(url, headers=headers, params=params)
#         r.raise_for_status()
#         data = r.json()
#         offres = data.get("resultats", [])
#         for o in offres:
#             all_jobs.append({
#                 "source": "France Travail",
#                 "titre": o.get("intitule"),
#                 "entreprise": o.get("entreprise", {}).get("nom"),
#                 "lieu": o.get("lieuTravail", {}).get("libelle"),
#                 "date": o.get("dateCreation"),
#                 "url": o.get("origineOffre", {}).get("urlOrigine"),
#             })
#     return all_jobs

# # ---------------------------
# # MAIN
# # ---------------------------
# if __name__ == "__main__":
#     print("Authentification France Travail...")
#     token = get_ft_token()

#     print("Récupération des offres France Travail...")
#     ft_jobs = fetch_france_travail_jobs(token)

#     df = pd.DataFrame(ft_jobs)
#     display(df.head())

#     # Export vers CSV
#     df.to_csv("../data/raw_data/offres_emploi.csv", index=False, encoding="utf-8")
#     print(f"{len(ft_jobs)} offres uniques exportées dans offres_emploi.csv ✅")
